# Advisor Transaction Synthesis

By importing Indicator Name and Value data from the Investment Data Atlas, we can synthesise RIA Transaction Data by randomly adding a specific RIA to each transaction. Then by aggregating the data we can predict which RIA's are better suited to each sector.



Table used: INVESTMENT_DATA_ATLAS.INVESTMENT."IMFCDIS2017" from Snowflake Data Atlas

## Data Synthesis

In [163]:
import pandas as pd
import numpy as np
from random import choices
import statistics

In [164]:
# Read in the Data Frame

df = pd.read_csv("BD and RIA Firm_Rep Data (Sample) - Examples.csv")
# df.head(n=30)

In [165]:
# Remove Outward and Inward after adding in negative values

Investment_Positions = pd.unique(df['Indicator Name'])

for i, row in df.iterrows():
    df.loc[i, 'Indicator Name'] = df.loc[i, 'Indicator Name'] \
            .replace(', Derived','')
    if "Outward" in row['Indicator Name']:
        df.loc[i, 'Value'] = -1*(df.loc[i, 'Value'])
        df.loc[i, 'Indicator Name'] = df.loc[i, 'Indicator Name'] \
            .replace('Outward','')
    else:
        df.loc[i, 'Indicator Name'] = df.loc[i, 'Indicator Name'] \
            .replace('Inward','')

## Analysis

In [166]:
# Randomly assign a sample RIA with equal probability to each transaction
improvements = []
for i in range(0,100):
    RIA_assignment = choices(range(1,100) ,k=len(df))
    df['RIA'] = RIA_assignment

    # Analysis
    df2 = df.groupby(['RIA', 'Indicator Name'])['Value'].sum()

    #mins = df2.groupby(['Indicator Name']).min()
    maxs = df2.groupby(['Indicator Name']).max()
    means = df2.groupby(['Indicator Name']).mean()
    normalized = (maxs-(maxs+means)/2)/maxs
    improvements.append(normalized.mean())
print(normalized)
print("Mean Improvement over average investor: ", statistics.mean(improvements))

Indicator Name
 Debt Instruments Assets Positions (Gross)                                           0.713492
 Debt Instruments Liabilities Positions (Gross)                                      0.460979
 Debt Instruments Positions (Net)                                                    0.470804
 Debt Positions (Net): Resident Enterprises that are not Financial Intermediaries    0.473061
 Debt Positions (Net): Resident Financial Intermediaries                             0.435573
 Direct Investment Assets Positions (Gross) with Fellow Enterprises                  0.824144
 Direct Investment Liabilities Positions (Gross) with Fellow Enterprises             0.569223
 Direct Investment Positions                                                         0.534609
 Direct Investment Positions (Net) with Fellow Enterprises                           0.437161
 Equity Positions (Net)                                                              0.536850
Name: Value, dtype: float64
Mean Improvement 

From these values we can clearly see that the very best investors generate a lot more value than the average investor. Most advisors are probably better than the average investor, but there is likely also a large difference between advisors' portfolios that we cannot observe without more data.